# 02. First Steps with LangChain

In [ ]:
import os

import dotenv
from langchain_openai import ChatOpenAI


class Config:
    def __init__(self):
        # By default, load_dotenv doesn't override existing environment variables and looks for a .env file in same directory as python script or searches for it incrementally higher up.
        dotenv_path = dotenv.find_dotenv(usecwd=True)
        if not dotenv_path:
            raise ValueError("No .env file found")
        dotenv.load_dotenv(dotenv_path=dotenv_path)

        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY is not set")

        base_url = os.getenv("OPENAI_API_BASE_URL")
        if not base_url:
            raise ValueError("OPENAI_API_BASE_URL is not set")

        model = os.getenv("OPENAI_MODEL")
        if not model:
            raise ValueError("OPENAI_MODEL is not set")

        vl_model = os.getenv("OPENAI_VL_MODEL")

        self.api_key = api_key
        self.base_url = base_url
        self.model = model
        self.vl_model = vl_model

    def new_openai_like(self, **kwargs) -> ChatOpenAI:
        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=self.model, **kwargs
        )

    def new_openai_like_vl(self, **kwargs) -> ChatOpenAI:
        if not self.vl_model:
            raise ValueError("OPENAI_VL_MODEL is not set")

        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=self.vl_model, **kwargs
        )

## Running local models

### Getting started with Ollama

In [ ]:
!bash ollama.sh up

In [ ]:
!bash ollama.sh run deepseek-r1:1.5b

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Ollama with your chosen model
local_llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0,
    # 将 base_url 的地址部分替换为上一步打印出来的地址。
    base_url="http://172.17.0.4:11434",
)
# Create an LCEL chain using the local model
prompt = PromptTemplate.from_template("Explain {concept} in simple terms")
local_chain = prompt | local_llm | StrOutputParser()
# Use the chain with your local model
result = local_chain.invoke({"concept": "quantum computing"})
print(result)

In [ ]:
!bash ollama.sh down

### Working with Hugging Face models locally

In [ ]:
%uv pip install langchain-huggingface transformers torch==2.8.0

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

# Create a pipeline with a small model:
llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)
chat_model = ChatHuggingFace(llm=llm)
# Use it like any other LangChain LLM
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Explain the concept of machine learning in simple terms"),
]
ai_msg = chat_model.invoke(messages)
print(ai_msg.content)


### Tips for local models

In [ ]:
import time

def safe_model_call(llm, prompt, max_retries=2):
    """Safely call a local model with retry logic and graceful failure"""
    retries = 0
    while retries <= max_retries:
        try:
            return llm.invoke(prompt)
        except RuntimeError as e:
            # Common error with local models when running out of VRAM
            if "CUDA out of memory" in str(e):
                print(
                    f"GPU memory error, waiting and retrying ({retries+1}/{max_retries+1})"
                )
                time.sleep(2)  # Give system time to free resources
                retries += 1
            else:
                print(f"Runtime error: {e}")
                return "An error occurred while processing your request."
        except Exception as e:
            print(f"Unexpected error calling model: {e}")
            return "An error occurred while processing your request."
    # If we exhausted retries
    return "Model is currently experiencing high load. Please try again later."


# Use the safety wrapper in your LCEL chain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

llm = Config().new_openai_like()

prompt = PromptTemplate.from_template("Explain {concept} in simple terms")
safe_llm = RunnableLambda(lambda x: safe_model_call(llm, x))
safe_chain = prompt | safe_llm
response = safe_chain.invoke({"concept": "quantum computing"})
print(response)


## Multimodal AI applications
### Text-to-image
#### Using DALL-E through OpenAI
TODO：找到或实现支持 langchain 接口的千义同问文生图模型（候选 qwen-image-plus）。

### Image understanding
#### Using Qwen-VL (similar to Gemini 1.5 Pro)

In [ ]:
import base64

from langchain_core.messages.human import HumanMessage

with open("static/stable-diffusion.png", "rb") as image_file:
    image_bytes = image_file.read()
    base64_bytes = base64.b64encode(image_bytes).decode("utf-8")


prompt = [
    {"type": "text", "text": "Describe the image: "},
    {
        "type": "image_url",
        "image_url": {"url": f"data:image/jpeg;base64,{base64_bytes}"},
    },
]


# initialize OpenAI-like model
llm = Config().new_openai_like_vl()
response = llm.invoke([HumanMessage(content=prompt)])
print(response.content)

In [ ]:
import base64

from langchain_core.messages import HumanMessage
from utils import Config


def analyze_image(image_path: str, question: str) -> str:
    # chat = ChatOpenAI(model="gpt-4o-mini", max_tokens=256)
    chat = Config().new_openai_like_vl(max_tokens=256)

    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
        base64_bytes = base64.b64encode(image_bytes).decode("utf-8")

    message = HumanMessage(
        content=[
            {"type": "text", "text": question},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_bytes}",
                    "detail": "auto",
                },
            },
        ]
    )

    response = chat.invoke([message])
    return response.content


# Example usage
image_path = "static/skyscrapers.png"
questions = [
    "What objects do you see in this image?",
    "What is the overall mood or atmosphere?",
    "Are there any people in the image?",
]

for question in questions:
    print(f"\nQ: {question}")
    print(f"A: {analyze_image(image_path, question)}")